# Term Structure & Black IR Demo

This notebook demonstrates OIS/IRS curve bootstrapping and Black IR caps/floors pricing with DV01/PV01.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from derivx.rates.term_structure import build_zero_curve
from derivx.rates.black_ir import price_cap_floor
from derivx.risk.metrics import scen_shift_curve

%matplotlib inline


## Bootstrap zero curve from market rates


In [ ]:
# Market quotes (OIS/IRS)
quotes = {
    "1W": 0.0512, "1M": 0.0520, "3M": 0.0525,
    "6M": 0.0528, "1Y": 0.0530, "2Y": 0.0535,
    "5Y": 0.0550, "10Y": 0.0560
}

curve = build_zero_curve(quotes)
print(f"Built curve with {len(curve.ts)} tenors")

# Plot discount factors
plt.figure(figsize=(10, 5))
plt.plot(curve.ts, curve.dfs, marker='o', linewidth=2)
plt.xlabel('Time (years)')
plt.ylabel('Discount Factor')
plt.title('Zero Curve Discount Factors')
plt.grid(True, alpha=0.3)
plt.show()


## Price a cap and compute PV/DV01/PV01


In [ ]:
# Cap parameters
K = 0.03  # 3% strike
tenors = np.linspace(0.5, 5.0, 10)
flat_vol = 0.20
vol_curve = type(curve)(ts=curve.ts, dfs=np.full_like(curve.dfs, flat_vol))

# Price cap
res = price_cap_floor(curve, K, tenors, vol_curve)
print(f"Cap PV: {res['PV']:.6f}")
print(f"DV01: {res['DV01']:.6f}")
print(f"PV01: {res['PV01']:.6f}")
print("\nBuckets (tenor: caplet PV):")
for t, pv in list(res['buckets'].items())[:5]:
    print(f"  {t:.2f}Y: {pv:.6f}")


## Scenario analysis: parallel shift


In [ ]:
# Parallel shift scenarios
shifts = [-100, -50, 0, 50, 100]
pvs = []

for shift_bp in shifts:
    shifted_curve = scen_shift_curve(curve, parallel_bp=shift_bp)
    res_shift = price_cap_floor(shifted_curve, K, tenors, vol_curve)
    pvs.append(res_shift['PV'])

# Plot scenario P&L
plt.figure(figsize=(10, 5))
plt.plot(shifts, pvs, marker='o', linewidth=2)
plt.axhline(res['PV'], color='red', linestyle='--', alpha=0.5, label='Base PV')
plt.xlabel('Parallel Shift (bp)')
plt.ylabel('Cap PV')
plt.title('Scenario Analysis: Parallel Rate Shifts')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Base PV: {res['PV']:.6f}")
print(f"PV at +100bp: {pvs[-1]:.6f} (delta: {pvs[-1] - res['PV']:.6f})")
